## Importar paqueterías necesarias

In [1]:
from imp import reload
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import setup_analysis as sa

In [ ]:
reload(sa)

## Lectura de datos
En esta sección se hace la lectura de los cuatro archivos que contienen las respuestas a las distintas secciones de la ENUT y se guardan como las variables <code>tvivienda</code>, <code>thogar</code>, <code>tsdem</code> y <code>tmodulo</code>.

A continuación, se hace la lectura del archivo que contiene las variables seleccionadas de la encuesta. Se modifican también las variables de la columna <code>variable</code> para empatar el formato de las otras tablas. Se obtiene como resultado la variable <code>var_sel</code>, la cual incluye información sobre las variables deseadas.

In [2]:
tvivienda = pd.read_csv(sa.tvivienda)
thogar = pd.read_csv(sa.thogar)
tsdem = pd.read_csv(sa.tsdem)
tmodulo = pd.read_csv(sa.tmodulo)

In [3]:
var_sel = pd.read_csv(sa.var_sel)
var_sel.head()

,encuesta,variable,tabla,pregunta,elemento
0,ENUT,UPM,thogar,Número de UPM,Identificación
1,ENUT,VIV_SEL,thogar,Número de vivienda seleccionada,Identificación
2,ENUT,HOGAR,thogar,Número de Hogar,Identificación
3,ENUT,N_INF,thogar,Número de renglón del informante,Identificación
4,ENUT,p2_6_1,thogar,¿En este hogar contratan trabajadoras(es) domé...,Trabajadores en el hogar


In [4]:
var_sel["variable"] = var_sel["variable"].str.upper()
var_sel.head()

,encuesta,variable,tabla,pregunta,elemento
0,ENUT,UPM,thogar,Número de UPM,Identificación
1,ENUT,VIV_SEL,thogar,Número de vivienda seleccionada,Identificación
2,ENUT,HOGAR,thogar,Número de Hogar,Identificación
3,ENUT,N_INF,thogar,Número de renglón del informante,Identificación
4,ENUT,P2_6_1,thogar,¿En este hogar contratan trabajadoras(es) domé...,Trabajadores en el hogar


Se seleccionan de las tablas importadas únicamente aquellas variables incluidas en la tabla <code>var_sel</code>.

In [5]:
vars_viv = list(var_sel["variable"][var_sel["tabla"]=="tvivienda"])
vars_hog = list(var_sel["variable"][var_sel["tabla"]=="thogar"])
vars_sdm = list(var_sel["variable"][var_sel["tabla"]=="tsdem"])
vars_mod = list(var_sel["variable"][var_sel["tabla"]=="tmodulo"])

tvivienda = tvivienda[vars_viv]
thogar = thogar[vars_hog]
tsdem = tsdem[vars_sdm]
tmodulo = tmodulo[vars_mod]

<code>tmodulo</code> es una de las tablas con observaciones individuales, por lo que se espera que sea una de las tablas con mayor número de valores, y a partir del cual se pueden unir tablas fácilmente. Al usar la función <code>describe()</code>, se observa que todas las variables observables son numéricas. Sin embargo, se desea generar un identificador para los registros compuesto por las variables <code>UPM</code>, <code>VIV_SEL</code>, <code>HOGAR</code>, y <code>N_REN</code>. Este identificador contendrá esta información separada por guiones bajos, por lo cual estas variables deben ser transformadas a string. La tabla <code>tsdem</code> también incluye datos de individuos, por lo que se le tratará de una manera similar.

In [6]:
tmodulo.describe()

,UPM,ENT,VIV_SEL,HOGAR,N_REN,P5_1,P5_3_1,P5_3_3,P5_5,P5_6_1,...,P6_15_4,P6_15A_4_1,P6_15A_4_2,P6_15A_4_3,P6_15A_4_4,P6_16_6,P6_16A_6_1,P6_16A_6_2,P6_16A_6_3,P6_16A_6_4
count,71404.000000,71404.000000,71404.000000,71404.000000,71404.000000,71404.000000,44028.000000,44028.000000,44657.000000,30208.000000,...,15994.000000,4005.000000,4005.000000,4005.000000,4005.000000,71404.000000,2179.000000,2179.000000,2179.000000,2179.000000
mean,1995.312489,16.630889,6.853860,1.018766,2.215968,1.413898,36.138912,7.112905,1.965134,1.529164,...,1.749594,14.055680,5.181273,6.627715,2.409488,1.969484,5.236347,4.690225,2.138596,1.763194
std,1086.300502,9.180151,5.680367,0.155964,1.364706,0.492534,16.851626,7.273909,1.345657,0.811704,...,0.433261,16.648093,12.311561,7.811112,8.219249,0.172005,8.012137,11.210604,4.467932,7.166196
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1072.000000,9.000000,3.000000,1.000000,1.000000,1.000000,25.000000,0.000000,1.000000,1.000000,...,1.000000,2.000000,0.000000,1.000000,0.000000,2.000000,1.000000,0.000000,0.000000,0.000000
50%,2015.000000,17.000000,5.000000,1.000000,2.000000,1.000000,40.000000,6.000000,1.000000,1.000000,...,2.000000,7.000000,0.000000,4.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000
75%,2910.000000,25.000000,11.000000,1.000000,3.000000,2.000000,47.000000,10.000000,4.000000,2.000000,...,2.000000,20.000000,0.000000,10.000000,0.000000,2.000000,6.000000,0.000000,2.000000,0.000000
max,3814.000000,32.000000,23.000000,5.000000,19.000000,2.000000,99.000000,48.000000,5.000000,9.000000,...,2.000000,99.000000,50.000000,48.000000,50.000000,2.000000,60.000000,50.000000,45.000000,50.000000


In [7]:
tmodulo[["VIV_SEL","HOGAR","N_REN","UPM"]] = tmodulo[["VIV_SEL","HOGAR","N_REN","UPM"]].astype("string")
tmodulo["ENT"] = pd.Categorical(tmodulo["ENT"])
tmodulo["uid"] = tmodulo["UPM"] + "_" + tmodulo["VIV_SEL"] + "_" + tmodulo["HOGAR"] + "_" + tmodulo["N_REN"]

tsdem[["VIV_SEL","HOGAR","N_REN","UPM"]] = tsdem[["VIV_SEL","HOGAR","N_REN","UPM"]].astype("string")
tsdem["uid"] = tsdem["UPM"] + "_" + tsdem["VIV_SEL"] + "_" + tsdem["HOGAR"] + "_" + tsdem["N_REN"]

Se hará el experimento de unir las tablas <code>tmodulo</code> y <code>tsdem</code>. Para esto, es necesario conocer cuál tabla es más grande, de lo contrario, es posible realizar un cruce erróneo y eliminar datos. En este caso, la tabla <code>tsdem</code> es de mayor longitud, por lo que se realizará un left join con esta tabla como la tabla izquierda.

In [ ]:
print(len(tsdem["UPM"]))
len(tmodulo["UPM"])

Al realizar el cruce, la longitud de la nueva tabla es la misma que la de <code>tsdem</code>, lo que significa que el cruce se realizó de manera correcta.

In [ ]:
bd = tsdem.merge(tmodulo, 'left',['uid'])
print(len(bd["uid"]))
len(bd)

La siguiente tabla con mayor desagregación es <code>thogar</code>.

In [ ]:
thogar[["VIV_SEL","HOGAR","UPM"]] = thogar[["VIV_SEL","HOGAR","UPM"]].astype("string")
thogar["hid"] = thogar["UPM"] + "_" + thogar["VIV_SEL"] + "_" + thogar["HOGAR"]

# Longitudes de base de datos y lista de identificadores de hogar.
print(len(thogar))
print(len(thogar["hid"].unique()))

In [ ]:
bd = bd.rename(columns={'UPM_x':'UPM','VIV_SEL_x':'VIV_SEL', 'HOGAR_x':'HOGAR', 'N_REN_x':'N_REN'})

In [ ]:
bd = bd.merge(thogar,  how='left', on=['UPM','VIV_SEL','HOGAR'])

In [ ]:
bd

In [ ]:
bd.to_csv(os.path.join(sa.dir_products,"csv","bd.csv"))

In [ ]:
bd["P3_11"]